# Lấy bias của từng action

In [109]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [111]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)


[0.93416695 0.95055215 1.         0.99461681 0.9098014  0.90772496
 0.92009333 0.88449282 0.90616824 0.9128886  0.89605652 0.87289172
 0.93425995 0.91965533 0.87920116]


In [117]:
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

518


# Bài tập 1 : Tìm ra feature hoặc bộ feature cho từng action để tỉ lệ +value khi active = 1 


In [173]:
def get_feature(i):
  #State old là state Hiện tại 
  #State new là state Mới 
  rate = 0 
  while rate !=1 :
    old = np.load('old'+f'{i}'+'.npy')
    new = np.load('new'+f'{i}'+'.npy')
    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.random.uniform(-1,1,(161,1))
    #state mà kích hoạt (1,0)
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
    rate = (sum(y)/sum(k))
  return l,rate,sum(y)
matrix , rate , sum = (get_feature(0))
matrix =  matrix.reshape(1,161)
print(rate)
print(sum)

[1.]
[4933.]


# Bài tập 2 : Khi dùng bộ feature đó cho từng action, tỉ lệ thắng >35% 


In [175]:
perx=[0]
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = matrix@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 0 
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p0, 1000,perx,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 26.6


In [ ]:
#State old là state Hiện tại 
#State new là state Mới 
old = np.load('old0.npy')
new = np.load('new0.npy')
# state mà có cả Value của old and new (1,0)
o = (new>old)
o= 1*o
i = np.zeros((245142,1))
for u in range(245142) :
    if sum(o[u])>0 :
        # print(o[u])
        i[u] = 1
#  a matrix with random value
l = np.random.uniform(-1,1,(161,1))
#state mà kích hoạt (1,0)
k = 1*((new@l)>=0)
# state mà kích hoạt và có reward (1,0)
y = np.zeros((245142,1))
for p in range(245142):
    if sum(k[p]) >= 1 and sum(i[p])>= 1:
        y[p] = 1
    else:
        y[p] = 0
        
print(sum(y)/sum(k))
# print(old.shape)
print(sum(y))
l = l.reshape(1,161)

#  feature cho các action 1


In [118]:
old1 = np.load('old1.npy')
new1 = np.load('new1.npy')
# state mà có cả Value của old and new (1,0)
o1 = (new1>old1)
o1= 1*o1
i1 = np.zeros((245378,1))
for u1 in range(245378) :
    if sum(o1[u1])>0 :
        i1[u1] = 1

        
#  a matrix with random value
l1 = np.random.uniform(-1,1,(161,1))

#state mà kích hoạt (1,0)
k1 = 1*((new1@l1)>=0)
# state mà kích hoạt và có reward (1,0)
y1 = np.zeros((245378,1))
for p1 in range(245378):
    if sum(k1[p1]) >= 1 and sum(i1[p1])>= 1:
        y1[p1] = 1
    else:
        y1[p1] = 0
print(sum(y1)/sum(k1))
# print(old.shape)
print(sum(y1))
l1 = l1.reshape(1,161)

[0.9491954]
[171176.]


In [169]:
@njit
def p_action1(state,per):
    actions = getValidActions(state)
    matran1 = l1@state    
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    if matran1 == 1 :
        action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_action1, 10000,perx,0)
print('tỷ lệ thắng',(win/10000)*100)

tỷ lệ thắng 37.62


# Lấy bias của từng action

In [ ]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [ ]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)


[0.93416695 0.95055215 1.         0.99461681 0.9098014  0.90772496
 0.92009333 0.88449282 0.90616824 0.9128886  0.89605652 0.87289172
 0.93425995 0.91965533 0.87920116]


In [ ]:
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

518


# Ghép các action với nhau 

In [120]:
perx1 = [bias_greedy]

In [122]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = l1@state   
    matran2 = l@state     
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    for i in range(len(matran2)):
        matran2[i] = int(matran2[i] >= 0)

    if matran1 == 1 and matran2 == 0:
        action = 1
    elif matran2 == 1 and matran1 == 0 :
        action = 0
    elif matran2 == matran1 and matran1 == 1:
        if per[0][0]+1 > per[0][1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1 or action == 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 1000,perx1,0)
print('tỷ lệ thắng',(win/1000)*100)

tỷ lệ thắng 41.199999999999996


# Homework 1:

*  **1.For each action , save at least 200k state both old and new state** 
*  **2.Label the state  which change 0 and 1** 
*  **3.Predict what state will get reward** 




**1.For each action , save at least 200k state both old and new state** 

In [ ]:
import numpy as np
from base.Splendor_v3.env import *
for k in range(2,16):
  @njit()
  def p0(state,per):
    # Choose an action
    actions = getValidActions(state)
    actions = np.where(actions == 1)[0]
    action = actions[np.random.randint(len(actions))]
    # Lưu new state
    if  per[3][0][0] == 1: 
      if int(per[4][0][0])<=10:
        j = int(per[4][0][0])
        for i in range(getStateSize()):
          per[2][j][i] += state[i]
        print('lưu state sau khi action '+f'{k}',per[2][j])
        # trigger action 2
        per[3][0][0] = 0
        per[4][0][0] += 1
        print("action sau khi action "+f'{k}',action)

    #Lưu old state 
    if action == k :
      if int(per[1][0][0])<=10:
        # save state
        j = int(per[1][0][0])
        for i in range(getStateSize()):
          per[0][j][i] += state[i]
        print('lưu state trước khi action '+f'{k}',per[0][j])
        # count number of state
        per[1][0][0] += 1
        per[3][0][0] = 1
        print("action ",action)
      
    return action,per

          #0                                  #1                #2                              #3            #4
  perx = [np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((200000,getStateSize())),np.zeros((1,1)),np.zeros((1,1))]
            #state old                      #số trận của old       #state new                  # trigger      # số trận của new
  win, x = numba_main_2(p0, 10,perx , 0)
  np.save('old'+f'{k}'+'.npy',x[0])
  np.save('new'+f'{k}'+'.npy',x[2])

*  **2.Label the state  which change 0 and 1** 
*  **3.Predict what value in state will change** 

**Logistic regression**

In [46]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_img, test_img, train_lbl, test_lbl = train_test_split(old1, i1, test_size=1/7.0, random_state=0)

clf1 = LogisticRegression(random_state=0).fit(train_img, train_lbl)
clf1.score(test_img, test_lbl)


0.9730130655559993

In [92]:
old1 = np.load('old1.npy')
new1 = np.load('new1.npy')
# state mà có cả Value của old and new (1,0)
o1 = (new1>old1)
o1= 1*o1
i1 = np.zeros((245378,1))
for u1 in range(245378) :
    if sum(o1[u1])>0 :
        i1[u1] = 1

In [97]:
print(np.where(i1==0)[0])
print(old1[30])
print(new[30])


[    30     32     52 ... 245287 245348 245366]
[ 2.  1.  0.  0.  0.  3.  1.  3.  2.  3.  0.  0.  2.  1.  3.  4.  3.  6.
  2.  2.  0.  5.  3.  0.  0.  3.  3.  0.  0.  0.  6.  0.  2.  3.  0.  0.
  0.  0.  5.  2.  0.  0.  4.  2.  0.  1.  3.  3.  3.  3.  5.  0.  3.  5.
  3.  7.  0.  0.  3.  0.  3.  1.  3.  0.  3.  5.  3.  4.  3.  6.  0.  3.
  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  3.  3.  0.  0.  4.
  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  2.  1.  0.  3.  0.  0.  5.  2.  0.  0.  0.  0.  5.  3.  5.  0.  3.
  0.  7.  0.  0.  0.  0.  0.  0.  0.  8. 19.  5.  0.  1.  1.  8.  0.]
[ 0.  0.  0.  0.  1.  4.  1.  1.  2.  1.  2.  1.  1.  1.  3.  0.  0.  2.
  0.  1.  2.  0.  2.  0.  1.  0.  3.  0.  0.  3.  0.  0.  0.  0.  0.  0.
  1.  2.  2.  0.  3.  1.  2.  0.  0.  2.  2.  3.  3.  0.  5.  0.  0.  2.
  0.  0.  0.  0.  5.  0.  2.  4.  4.  0.  1.  2.  0.  1.  1.  3.  2.  2.
  0.  

In [106]:
print(clf1.predict(old1[52].reshape(1,161)))

[0.]


# Homework 2:
**Tìm bộ feature cho từng action sao cho**
* **(tỷ lệ state vừa kích hoạt vừa có value /tỷ lệ state vừa kích hoạt) luôn bằng 1**
* **check xem từng action tỷ lệ thắng có trên 30%**

# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

In [ ]:
@njit
def p_all_action(state,per):
    actions = getValidActions(state)
    matran1 = l1@state   
    matran2 = l@state     
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    for i in range(len(matran2)):
        matran2[i] = int(matran2[i] >= 0)

    if matran1 == 1 and matran2 == 0:
        action = 1
    elif matran2 == 1 and matran1 == 0 :
        action = 0
    elif matran2 == matran1 and matran1 == 1:
        if per[0][0]+1 > per[0][1]+1:
            action = 0
        else:
            action = 1
    else:
        actions = np.where(actions == 1)[0]
        action = actions[np.random.randint(len(actions))]
        while action== 1 or action == 0:
            action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p_all_action, 1000,perx1,0)
print('tỷ lệ thắng',(win/1000)*100)

In [154]:
# import numpy as np
# from numba import njit 
# #Khởi tạo weight 
# perx = [np.zeros((getActionSize()*5,getStateSize())),np.zeros((getActionSize(),getActionSize()*5))]
# for i in range(0,getActionSize()):
#     for j in range(5):
#         perx[0][i*5+j][6+j] = 1
#         perx[0][i*5+j][11] = 0 
#         perx[0][i*5+j][12+j] = 1
#         perx[0][i*5+j][20+i*7+j] = -1
# for i in range(0,getActionSize()):
#     for j in range(5):
#         perx[1][i][i*5+j] = 1

from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1:
        matran2 += bias_greedy
        # print(matran2)
        action = np.argmax(matran2)
    else:
        action = np.argmax(matran2)
    return action,per

win, x = numba_main_2(p0, 10000,perx,0)

In [156]:
print(win)
print(getActionSize())

9379
15
